In [3]:
# Libraries install & import

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, GRU, Bidirectional, Flatten, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import ModelCheckpoint, EarlyStopping

#from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import mean_squared_error
import math
from math import sqrt
from random import randint


np.random.seed(1356)
plt.style.use('ggplot')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# Read data from csv
df_btc = pd.read_csv("https://raw.githubusercontent.com/jraheja1994/CS_5661_Project/main/BTC-USD.csv", index_col='Date', parse_dates=['Date'])
print(df_btc)

HTTPError: ignored

In [ ]:
# Plot Bitcoin price trend

pltDataset = pd.DataFrame(df_btc['Close'])
plt.figure(figsize=(8,4))
plt.rcParams['figure.dpi'] = 360
plt.plot(pltDataset, linewidth=0.8)
plt.xlabel("Date", fontsize=8) 
plt.ylabel('Close (USD)', fontsize=6)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%m/%Y"))
plt.xticks(fontsize=8, rotation=90)
plt.yticks(fontsize=4)
plt.title("Bitcoin USD (BTC-USD)")

In [ ]:
plt.figure(figsize=(15,12))
plt.suptitle('Lag Plots', fontsize=22)

plt.subplot(3,3,1)
pd.plotting.lag_plot(df_btc['Close'], lag=1) #Daily lag
plt.title('Daily lag')

plt.subplot(3,3,2)
pd.plotting.lag_plot(df_btc['Close'], lag=7) #weekly lag
plt.title('Weekly Lag')

plt.subplot(3,3,3)
pd.plotting.lag_plot(df_btc['Close'], lag=30) #30 Days lag
plt.title('30 Day(s) Lag')

plt.subplot(3,3,4)
pd.plotting.lag_plot(df_btc['Close'], lag=365) #Yearly lag
plt.title('Yearly Lag')

plt.show()

In [ ]:
# Data Spliting
prediction_days = 20

df_train= pltDataset[:len(pltDataset)-prediction_days]
df_test= pltDataset[len(pltDataset)-prediction_days:]

# Some preprocessing, like normalizing the values, reshaping, etc. The data is a time series data, so the output to every instance is the next instance. 
training_df = df_train.values
training_df = np.reshape(training_df, (len(training_df), 1))
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_df = sc.fit_transform(training_df)
X_train = training_df[0:len(training_df)-1]
y_train = training_df[1:len(training_df)]
X_train = np.reshape(X_train, (len(X_train), 1, 1))

print(training_df.shape)
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Plot Train & Test Data
plt.figure(figsize=(8,4))
plt.rcParams['figure.dpi'] = 360
plt.plot(df_train, label="Train Data", linewidth=0.8)
plt.plot(df_test, label="Test Data", linewidth=0.8)
plt.xlabel("Date", fontsize=8) 
plt.ylabel('Close (USD)', fontsize=6)
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%m/%Y"))
plt.xticks(fontsize=8, rotation=90)
plt.yticks(fontsize=4)
plt.title("Train & Test Data")

In [ ]:
# LSTM model

model = Sequential()
model.add(LSTM(units = 4, activation = 'sigmoid', input_shape = (None, 1)))
model.add(Dense(units = 1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(X_train, y_train, batch_size = 32, epochs = 100)

# Making the predictions
test_set = df_test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_BTC_price = model.predict(inputs)
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)

# MNB = MultinomialNB()
# MNB.fit(X_train, y_train)
# predictions = MNB.predict(inputs)
# print(classification_report(y_test, predictions))
# print(confusion_matrix(y_test, predictions))

# MNB_f1 = round(f1_score(y_test, predictions, average='weighted'), 3)
# MNB_accuracy = round((accuracy_score(y_test, predictions)*100),2)

# print("Accuracy : " , MNB_accuracy , " %")
# print("f1_score : " , MNB_f1)

# Mean Square Error
er1 = np.mean((predicted_BTC_price-test_set)**2)
print("Mean Square Error (MSE) using LSTM:", er1)

In [ ]:
test_set = df_test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_BTC_price = model.predict(inputs)
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)
er1 = np.mean((predicted_BTC_price-test_set)**2)
print("Mean Square Error (MSE) using LSTM:", er1)
rmse_lstm = math.sqrt(er1)
print(rmse_lstm)

In [ ]:
# Visualising the results

plt.figure(figsize=(25,15), dpi=80, facecolor='w', edgecolor='k')
ax = plt.gca()  
plt.plot(test_set, color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price, color = 'blue', label = 'Predicted BTC Price')
plt.title('Using LSTM', fontsize=40)
df_test = df_test.reset_index()
x = df_test.index
labels = df_test['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
plt.xticks(x, labels, rotation = 'vertical')
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 25})
plt.show()

In [ ]:
# LSTM model

model2 = Sequential()
model2.add(GRU(units = 4, activation = 'sigmoid', input_shape = (None, 1)))
model2.add(Dense(units = 1))
model2.compile(optimizer = 'adam', loss = 'mean_squared_error')
model2.fit(X_train, y_train, batch_size = 32, epochs = 100)


# Prediction from the trained GRU network
predicted_BTC_price = model2.predict(inputs)
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)
er2 = np.mean((predicted_BTC_price-test_set)**2)
print("Mean Square Error (MSE) using GRU:", er2)
rmse_gru = math.sqrt(er2)
print(rmse_gru)

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), dpi=80, facecolor='w', edgecolor='k')
ax = plt.gca()  
plt.plot(test_set, color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price, color = 'blue', label = 'Predicted BTC Price')
plt.title('Using Gated Recurrent Unit (GRU)', fontsize=40)
df_test = df_test.reset_index()
x=df_test.index
labels = df_test['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
plt.xticks(x, labels, rotation = 'vertical')
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 25})
plt.show()

In [ ]:
model3 = Sequential()
model3.add(Bidirectional(LSTM(units = 4, return_sequences=True), input_shape=(None, 1)))
model3.add(Bidirectional(LSTM(units = 4)))
model3.add(Dense(1))
#Compile model3
model3.compile(optimizer = 'adam', loss = 'mean_squared_error')
model3.fit(X_train, y_train, batch_size = 32, epochs = 100)

# Prediction from the trained GRU network
predicted_BTC_price = model3.predict(inputs)
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)
er3 = np.mean((predicted_BTC_price-test_set)**2)
print("Mean Square Error (MSE) using GRU:", er3)
rmse_bilstm = math.sqrt(er3)
print(rmse_bilstm)

In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), dpi=80, facecolor='w', edgecolor='k')
ax = plt.gca()  
plt.plot(test_set, color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price, color = 'blue', label = 'Predicted BTC Price')
plt.title('Using bi-Long Short Term Memory (bi-LSTM)', fontsize=40)
df_test = df_test.reset_index()
x=df_test.index
labels = df_test['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))
plt.xticks(x, labels, rotation = 'vertical')
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 25})
plt.show()